In [2]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

SEED = 0
pd.set_option('display.max_colwidth', None)
#nltk.download('wordnet')

## Load and view data

In [5]:
path = 'ai-medical-chatbot.csv'
df = pd.read_csv(path)

In [6]:
df.sample(5, random_state=SEED)

,Description,Patient,Doctor
231878,Suggest treatment for chest infection,"Yes, I have had a sinus infection for 2 weeks. My GP prescribed an antibiotic (Azithromycin) and Prednisone. I have finished both but my symptoms have changed and gotten worse. My chest is congested and I am coughing more with greenish mucus. What should I take now?","Thanks for your question on HCM.I can understand your situation and problem.In my opinion you should consult pulmonologist and get done1. Repeat chest x ray2. Sputum culture and sensitivity.You have taken treatment (antibiotics) and still not improving, rather worsening, so we need to rule out worsening of infection by chest x ray.And in pneumonia, we should first identify the causative organism and than guide antibiotic therapy according to sensitivity report. So sputum culture and sensitivity report is needed to titrate antibiotics.Haphazard use of antibiotics should be avoided in pneumonia as it will increase drug resistance and delay the recovery.So consult pulmonologist and start appropriate antibiotics according to sensitivity report."
63852,What causes pain in groin?,"Hello, My husband was complaining earlier of a painful boil in the groin, also he had pain higher up in the groin, away from the boil. He s gone to bed, and was only there 10 mins when he said he felt hot, but was shivering visibly. The feeling of unwell has come on him so quickly, and that s my concern.Can you help?","Hi,It seems that there might be having ingrown hair follicle infection giving rise enalrged inguinal lymph node in the groin.Due to bacterial infection he got shivering, feeling hot and unwell.Go for one antibiotic medicine like Amoxicilin for 3-5 days.Give analgesic like Ibuprofen as needed.Within 2-3 days he will be alright.Ok and take care."
45614,What causes pain in legs and abdomen?,I just had a baby a month and week ago and I have been having pain in my legs and abdomen. I also have had very thick brown and bloody discharge and when I pull it off I can feel it come out of me it is so thick. And also a increase in headaches. What is wrong with me?,"cannot say in your particular case, but generally, continuing pain and discharge a month later is a serious emergency requiring immediate attention."
157278,What makes an infant to puke after eating and pass fish odor stools?,Hi my son is 14 months and every time he eats food he poops it out within 5-10 mins and his stool has a fishy odor to it. The texture of his stool is liquid and you can still see the food as if it was not digested at all. The color is a grey cloudy color. Is this normal?,"Many thanks for consultation.It is not worrisome if he is active and alert.It could be due to tooth eruption and pain.they can get loose stool,vomits and fever due to this.please check for tooth eruption and if new tooth is coming up then buy some custom made toys for teeth eruption.paracetamol syrup for pain and irritation.Lastly if problem persist then visit to your local pediatric physician."
244134,Suggest treatment for chronic itching with rashes,"I am a 66 year old female suffering from a chronic itching with a rash. I have had this condition for approx. 8 weeks. I have tried many OTC remedies including calamine lotion, allegra, etc. I have also tried a 10 day dosage of Predasone 5mg without any relief. I have changed my laundry detergent and also discontinued certain prescription drugs temporarily. Nothing helps.","Hi, It may be lichen simplex chronicus most probably. Consult the dermatologist for the perfect diagnosis and proper treatment. If it is same as I said, I would recommend you to apply steroid cream like Clobetasole Propionate cream. And you should take steroid tablet in a tapering dose. If needed, an intradermal steroid injection may be taken. Take antihistaminics like Desloratadine. Avoid soap bath. Avoid dietary stuff like cows milk, fish, prawns, chicken, eggs, fermented food, nuts, soy products etc. Hope I have answered your query. Let me know i

# Preprocessing

#### Renaming columns

In [7]:
df = df.rename(columns={'Description': 'description',
                        'Patient': 'question',
                        'Doctor': 'answer'})

In [8]:
df.sample(5, random_state=SEED)

,description,question,answer
231878,Suggest treatment for chest infection,"Yes, I have had a sinus infection for 2 weeks. My GP prescribed an antibiotic (Azithromycin) and Prednisone. I have finished both but my symptoms have changed and gotten worse. My chest is congested and I am coughing more with greenish mucus. What should I take now?","Thanks for your question on HCM.I can understand your situation and problem.In my opinion you should consult pulmonologist and get done1. Repeat chest x ray2. Sputum culture and sensitivity.You have taken treatment (antibiotics) and still not improving, rather worsening, so we need to rule out worsening of infection by chest x ray.And in pneumonia, we should first identify the causative organism and than guide antibiotic therapy according to sensitivity report. So sputum culture and sensitivity report is needed to titrate antibiotics.Haphazard use of antibiotics should be avoided in pneumonia as it will increase drug resistance and delay the recovery.So consult pulmonologist and start appropriate antibiotics according to sensitivity report."
63852,What causes pain in groin?,"Hello, My husband was complaining earlier of a painful boil in the groin, also he had pain higher up in the groin, away from the boil. He s gone to bed, and was only there 10 mins when he said he felt hot, but was shivering visibly. The feeling of unwell has come on him so quickly, and that s my concern.Can you help?","Hi,It seems that there might be having ingrown hair follicle infection giving rise enalrged inguinal lymph node in the groin.Due to bacterial infection he got shivering, feeling hot and unwell.Go for one antibiotic medicine like Amoxicilin for 3-5 days.Give analgesic like Ibuprofen as needed.Within 2-3 days he will be alright.Ok and take care."
45614,What causes pain in legs and abdomen?,I just had a baby a month and week ago and I have been having pain in my legs and abdomen. I also have had very thick brown and bloody discharge and when I pull it off I can feel it come out of me it is so thick. And also a increase in headaches. What is wrong with me?,"cannot say in your particular case, but generally, continuing pain and discharge a month later is a serious emergency requiring immediate attention."
157278,What makes an infant to puke after eating and pass fish odor stools?,Hi my son is 14 months and every time he eats food he poops it out within 5-10 mins and his stool has a fishy odor to it. The texture of his stool is liquid and you can still see the food as if it was not digested at all. The color is a grey cloudy color. Is this normal?,"Many thanks for consultation.It is not worrisome if he is active and alert.It could be due to tooth eruption and pain.they can get loose stool,vomits and fever due to this.please check for tooth eruption and if new tooth is coming up then buy some custom made toys for teeth eruption.paracetamol syrup for pain and irritation.Lastly if problem persist then visit to your local pediatric physician."
244134,Suggest treatment for chronic itching with rashes,"I am a 66 year old female suffering from a chronic itching with a rash. I have had this condition for approx. 8 weeks. I have tried many OTC remedies including calamine lotion, allegra, etc. I have also tried a 10 day dosage of Predasone 5mg without any relief. I have changed my laundry detergent and also discontinued certain prescription drugs temporarily. Nothing helps.","Hi, It may be lichen simplex chronicus most probably. Consult the dermatologist for the perfect diagnosis and proper treatment. If it is same as I said, I would recommend you to apply steroid cream like Clobetasole Propionate cream. And you should take steroid tablet in a tapering dose. If needed, an intradermal steroid injection may be taken. Take antihistaminics like Desloratadine. Avoid soap bath. Avoid dietary stuff like cows milk, fish, prawns, chicken, eggs, fermented food, nuts, soy products etc. Hope I have answered your query. Let me know 

#### Converting the Descriptions, Questions, and Answers to lowercase

In [9]:
def to_lowercase(df: pd.DataFrame, cols) -> pd.DataFrame:
    for col in cols:
        df[col] = df[col].str.lower()
    return df
    
df = to_lowercase(df, df.columns)

In [10]:
df.sample(5, random_state=SEED)

,description,question,answer
231878,suggest treatment for chest infection,"yes, i have had a sinus infection for 2 weeks. my gp prescribed an antibiotic (azithromycin) and prednisone. i have finished both but my symptoms have changed and gotten worse. my chest is congested and i am coughing more with greenish mucus. what should i take now?","thanks for your question on hcm.i can understand your situation and problem.in my opinion you should consult pulmonologist and get done1. repeat chest x ray2. sputum culture and sensitivity.you have taken treatment (antibiotics) and still not improving, rather worsening, so we need to rule out worsening of infection by chest x ray.and in pneumonia, we should first identify the causative organism and than guide antibiotic therapy according to sensitivity report. so sputum culture and sensitivity report is needed to titrate antibiotics.haphazard use of antibiotics should be avoided in pneumonia as it will increase drug resistance and delay the recovery.so consult pulmonologist and start appropriate antibiotics according to sensitivity report."
63852,what causes pain in groin?,"hello, my husband was complaining earlier of a painful boil in the groin, also he had pain higher up in the groin, away from the boil. he s gone to bed, and was only there 10 mins when he said he felt hot, but was shivering visibly. the feeling of unwell has come on him so quickly, and that s my concern.can you help?","hi,it seems that there might be having ingrown hair follicle infection giving rise enalrged inguinal lymph node in the groin.due to bacterial infection he got shivering, feeling hot and unwell.go for one antibiotic medicine like amoxicilin for 3-5 days.give analgesic like ibuprofen as needed.within 2-3 days he will be alright.ok and take care."
45614,what causes pain in legs and abdomen?,i just had a baby a month and week ago and i have been having pain in my legs and abdomen. i also have had very thick brown and bloody discharge and when i pull it off i can feel it come out of me it is so thick. and also a increase in headaches. what is wrong with me?,"cannot say in your particular case, but generally, continuing pain and discharge a month later is a serious emergency requiring immediate attention."
157278,what makes an infant to puke after eating and pass fish odor stools?,hi my son is 14 months and every time he eats food he poops it out within 5-10 mins and his stool has a fishy odor to it. the texture of his stool is liquid and you can still see the food as if it was not digested at all. the color is a grey cloudy color. is this normal?,"many thanks for consultation.it is not worrisome if he is active and alert.it could be due to tooth eruption and pain.they can get loose stool,vomits and fever due to this.please check for tooth eruption and if new tooth is coming up then buy some custom made toys for teeth eruption.paracetamol syrup for pain and irritation.lastly if problem persist then visit to your local pediatric physician."
244134,suggest treatment for chronic itching with rashes,"i am a 66 year old female suffering from a chronic itching with a rash. i have had this condition for approx. 8 weeks. i have tried many otc remedies including calamine lotion, allegra, etc. i have also tried a 10 day dosage of predasone 5mg without any relief. i have changed my laundry detergent and also discontinued certain prescription drugs temporarily. nothing helps.","hi, it may be lichen simplex chronicus most probably. consult the dermatologist for the perfect diagnosis and proper treatment. if it is same as i said, i would recommend you to apply steroid cream like clobetasole propionate cream. and you should take steroid tablet in a tapering dose. if needed, an intradermal steroid injection may be taken. take antihistaminics like desloratadine. avoid soap bath. avoid dietary stuff like cows milk, fish, prawns, chicken, eggs, fermented food, nuts, soy products etc. hope i have answered your query. let me know 

#### Remove symbols and special characters from text

In [11]:
import re
def remove_non_alphanumeric(df: pd.DataFrame, cols) -> pd.DataFrame:
    
	for index, row in df.iterrows():
		for col in cols:
			df.at[index, col] = re.sub(r'[^\w]', ' ', df.at[index, col])

	return df
    
df = remove_non_alphanumeric(df, df.columns)

In [12]:
df.sample(5, random_state=SEED)

,description,question,answer
231878,suggest treatment for chest infection,yes i have had a sinus infection for 2 weeks my gp prescribed an antibiotic azithromycin and prednisone i have finished both but my symptoms have changed and gotten worse my chest is congested and i am coughing more with greenish mucus what should i take now,thanks for your question on hcm i can understand your situation and problem in my opinion you should consult pulmonologist and get done1 repeat chest x ray2 sputum culture and sensitivity you have taken treatment antibiotics and still not improving rather worsening so we need to rule out worsening of infection by chest x ray and in pneumonia we should first identify the causative organism and than guide antibiotic therapy according to sensitivity report so sputum culture and sensitivity report is needed to titrate antibiotics haphazard use of antibiotics should be avoided in pneumonia as it will increase drug resistance and delay the recovery so consult pulmonologist and start appropriate antibiotics according to sensitivity report
63852,what causes pain in groin,hello my husband was complaining earlier of a painful boil in the groin also he had pain higher up in the groin away from the boil he s gone to bed and was only there 10 mins when he said he felt hot but was shivering visibly the feeling of unwell has come on him so quickly and that s my concern can you help,hi it seems that there might be having ingrown hair follicle infection giving rise enalrged inguinal lymph node in the groin due to bacterial infection he got shivering feeling hot and unwell go for one antibiotic medicine like amoxicilin for 3 5 days give analgesic like ibuprofen as needed within 2 3 days he will be alright ok and take care
45614,what causes pain in legs and abdomen,i just had a baby a month and week ago and i have been having pain in my legs and abdomen i also have had very thick brown and bloody discharge and when i pull it off i can feel it come out of me it is so thick and also a increase in headaches what is wrong with me,cannot say in your particular case but generally continuing pain and discharge a month later is a serious emergency requiring immediate attention
157278,what makes an infant to puke after eating and pass fish odor stools,hi my son is 14 months and every time he eats food he poops it out within 5 10 mins and his stool has a fishy odor to it the texture of his stool is liquid and you can still see the food as if it was not digested at all the color is a grey cloudy color is this normal,many thanks for consultation it is not worrisome if he is active and alert it could be due to tooth eruption and pain they can get loose stool vomits and fever due to this please check for tooth eruption and if new tooth is coming up then buy some custom made toys for teeth eruption paracetamol syrup for pain and irritation lastly if problem persist then visit to your local pediatric physician
244134,suggest treatment for chronic itching with rashes,i am a 66 year old female suffering from a chronic itching with a rash i have had this condition for approx 8 weeks i have tried many otc remedies including calamine lotion allegra etc i have also tried a 10 day dosage of predasone 5mg without any relief i have changed my laundry detergent and also discontinued certain prescription drugs temporarily nothing helps,hi it may be lichen simplex chronicus most probably consult the dermatologist for the perfect diagnosis and proper treatment if it is same as i said i would recommend you to apply steroid cream like clobetasole propionate cream and you should take steroid tablet in a tapering dose if needed an intradermal steroid injection may be taken take antihistaminics like desloratadine avoid soap bath avoid dietary stuff like cows milk fish prawns chicken eggs fermented food nuts soy products etc hope i have answered your query let me know if i can assist you further take care regards dr ilyas patel dermatologist


#### Removing stopwords

In [16]:
stop_words = set(stopwords.words('english'))

# View some samples of stopwords to see what they look like
print(list(stop_words)[::12])

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\jgu04/nltk_data'
    - 'c:\\Users\\jgu04\\anaconda3\\nltk_data'
    - 'c:\\Users\\jgu04\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\jgu04\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\jgu04\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [15]:
def remove_stopwords(df: pd.DataFrame, cols) -> pd.DataFrame:
    
	count_removed = 0

	stop_words = set(stopwords.words('english'))

	for i, row in df.iterrows():
		for col in cols:

			# Get list of tokens in sentence
			sentence_tokens = word_tokenize(df.at[i, col])

			sentence_tokens_without_stopwords = []
			for token in sentence_tokens:
				if token not in stop_words:
					sentence_tokens_without_stopwords.append(token)
				else:
					count_removed += 1

			# Join list using spaces
			df.at[i, col] = " ".join(sentence_tokens_without_stopwords)

	print(f"Removed {count_removed} stopwords from the dataset")

	return df

df = remove_stopwords(df, ['description', 'question'])

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\jgu04/nltk_data'
    - 'c:\\Users\\jgu04\\anaconda3\\nltk_data'
    - 'c:\\Users\\jgu04\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\jgu04\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\jgu04\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [17]:
df.sample(5, random_state=SEED)

,description,question,answer
231878,suggest treatment for chest infection,yes i have had a sinus infection for 2 weeks my gp prescribed an antibiotic azithromycin and prednisone i have finished both but my symptoms have changed and gotten worse my chest is congested and i am coughing more with greenish mucus what should i take now,thanks for your question on hcm i can understand your situation and problem in my opinion you should consult pulmonologist and get done1 repeat chest x ray2 sputum culture and sensitivity you have taken treatment antibiotics and still not improving rather worsening so we need to rule out worsening of infection by chest x ray and in pneumonia we should first identify the causative organism and than guide antibiotic therapy according to sensitivity report so sputum culture and sensitivity report is needed to titrate antibiotics haphazard use of antibiotics should be avoided in pneumonia as it will increase drug resistance and delay the recovery so consult pulmonologist and start appropriate antibiotics according to sensitivity report
63852,what causes pain in groin,hello my husband was complaining earlier of a painful boil in the groin also he had pain higher up in the groin away from the boil he s gone to bed and was only there 10 mins when he said he felt hot but was shivering visibly the feeling of unwell has come on him so quickly and that s my concern can you help,hi it seems that there might be having ingrown hair follicle infection giving rise enalrged inguinal lymph node in the groin due to bacterial infection he got shivering feeling hot and unwell go for one antibiotic medicine like amoxicilin for 3 5 days give analgesic like ibuprofen as needed within 2 3 days he will be alright ok and take care
45614,what causes pain in legs and abdomen,i just had a baby a month and week ago and i have been having pain in my legs and abdomen i also have had very thick brown and bloody discharge and when i pull it off i can feel it come out of me it is so thick and also a increase in headaches what is wrong with me,cannot say in your particular case but generally continuing pain and discharge a month later is a serious emergency requiring immediate attention
157278,what makes an infant to puke after eating and pass fish odor stools,hi my son is 14 months and every time he eats food he poops it out within 5 10 mins and his stool has a fishy odor to it the texture of his stool is liquid and you can still see the food as if it was not digested at all the color is a grey cloudy color is this normal,many thanks for consultation it is not worrisome if he is active and alert it could be due to tooth eruption and pain they can get loose stool vomits and fever due to this please check for tooth eruption and if new tooth is coming up then buy some custom made toys for teeth eruption paracetamol syrup for pain and irritation lastly if problem persist then visit to your local pediatric physician
244134,suggest treatment for chronic itching with rashes,i am a 66 year old female suffering from a chronic itching with a rash i have had this condition for approx 8 weeks i have tried many otc remedies including calamine lotion allegra etc i have also tried a 10 day dosage of predasone 5mg without any relief i have changed my laundry detergent and also discontinued certain prescription drugs temporarily nothing helps,hi it may be lichen simplex chronicus most probably consult the dermatologist for the perfect diagnosis and proper treatment if it is same as i said i would recommend you to apply steroid cream like clobetasole propionate cream and you should take steroid tablet in a tapering dose if needed an intradermal steroid injection may be taken take antihistaminics like desloratadine avoid soap bath avoid dietary stuff like cows milk fish prawns chicken eggs fermented food nuts soy products etc hope i have answered your query let me know if i can assist you further take care regards dr ilyas patel dermatologist


#### Stemming

In [18]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def get_root_word(df: pd.DataFrame, cols) -> pd.DataFrame:
    
	for i, row in df.iterrows():
		for col in cols:

			# Get list of tokens in sentence
			sentence_tokens = word_tokenize(df.at[i, col])

			stemmed_tokens = []
			for token in sentence_tokens:
				stemmed_tokens.append(stemmer.stem(token))

			# Join list using spaces
			df.at[i, col] = " ".join(stemmed_tokens)

	return df

df = get_root_word(df, ['description', 'question'])


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\jgu04/nltk_data'
    - 'c:\\Users\\jgu04\\anaconda3\\nltk_data'
    - 'c:\\Users\\jgu04\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\jgu04\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\jgu04\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [19]:
df.sample(5, random_state=SEED)

,description,question,answer
231878,suggest treatment for chest infection,yes i have had a sinus infection for 2 weeks my gp prescribed an antibiotic azithromycin and prednisone i have finished both but my symptoms have changed and gotten worse my chest is congested and i am coughing more with greenish mucus what should i take now,thanks for your question on hcm i can understand your situation and problem in my opinion you should consult pulmonologist and get done1 repeat chest x ray2 sputum culture and sensitivity you have taken treatment antibiotics and still not improving rather worsening so we need to rule out worsening of infection by chest x ray and in pneumonia we should first identify the causative organism and than guide antibiotic therapy according to sensitivity report so sputum culture and sensitivity report is needed to titrate antibiotics haphazard use of antibiotics should be avoided in pneumonia as it will increase drug resistance and delay the recovery so consult pulmonologist and start appropriate antibiotics according to sensitivity report
63852,what causes pain in groin,hello my husband was complaining earlier of a painful boil in the groin also he had pain higher up in the groin away from the boil he s gone to bed and was only there 10 mins when he said he felt hot but was shivering visibly the feeling of unwell has come on him so quickly and that s my concern can you help,hi it seems that there might be having ingrown hair follicle infection giving rise enalrged inguinal lymph node in the groin due to bacterial infection he got shivering feeling hot and unwell go for one antibiotic medicine like amoxicilin for 3 5 days give analgesic like ibuprofen as needed within 2 3 days he will be alright ok and take care
45614,what causes pain in legs and abdomen,i just had a baby a month and week ago and i have been having pain in my legs and abdomen i also have had very thick brown and bloody discharge and when i pull it off i can feel it come out of me it is so thick and also a increase in headaches what is wrong with me,cannot say in your particular case but generally continuing pain and discharge a month later is a serious emergency requiring immediate attention
157278,what makes an infant to puke after eating and pass fish odor stools,hi my son is 14 months and every time he eats food he poops it out within 5 10 mins and his stool has a fishy odor to it the texture of his stool is liquid and you can still see the food as if it was not digested at all the color is a grey cloudy color is this normal,many thanks for consultation it is not worrisome if he is active and alert it could be due to tooth eruption and pain they can get loose stool vomits and fever due to this please check for tooth eruption and if new tooth is coming up then buy some custom made toys for teeth eruption paracetamol syrup for pain and irritation lastly if problem persist then visit to your local pediatric physician
244134,suggest treatment for chronic itching with rashes,i am a 66 year old female suffering from a chronic itching with a rash i have had this condition for approx 8 weeks i have tried many otc remedies including calamine lotion allegra etc i have also tried a 10 day dosage of predasone 5mg without any relief i have changed my laundry detergent and also discontinued certain prescription drugs temporarily nothing helps,hi it may be lichen simplex chronicus most probably consult the dermatologist for the perfect diagnosis and proper treatment if it is same as i said i would recommend you to apply steroid cream like clobetasole propionate cream and you should take steroid tablet in a tapering dose if needed an intradermal steroid injection may be taken take antihistaminics like desloratadine avoid soap bath avoid dietary stuff like cows milk fish prawns chicken eggs fermented food nuts soy products etc hope i have answered your query let me know if i can assist you further take care regards dr ilyas patel dermatologist


#### Saving Dataframe to file

In [16]:
df.to_csv('data/preprocessed-ai-medical-chatbot.csv', index=False)  